<a href="https://colab.research.google.com/github/kasodeep/inheritance-project/blob/main/llama_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install langchain
!pip install fpdf
!pip install ctransformers
!pip install pypdf
!pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [3]:
#imports
import langchain
from torch import cuda, bfloat16
from fpdf import FPDF
import torch
import transformers
from transformers import AutoTokenizer
from time import time
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader,PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain,ConversationalRetrievalChain,StuffDocumentsChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import CTransformers
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import PromptTemplate, LLMChain

C:\Users\MSHOME\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#loading downloaded llm suitable for local usage,temperature(entropy/randomness in answer):1e-2
# But first you would need to mount your Google Drive to Google Colab.
llm = CTransformers(model=r"/content/drive/MyDrive/llama-2-13b.ggmlv3.q2_K.bin", model_type="llama", streaming=True,
                    callbacks=[StreamingStdOutCallbackHandler()],
                    config={'max_new_tokens':4096,'temperature':0.01, 'context_length':4096})

ValidationError: 1 validation error for CTransformers
__root__
  Model path '/content/drive/MyDrive/llama-2-13b.ggmlv3.q2_K.bin' doesn't exist. (type=value_error)

In [ ]:
def load_documents(file_path):
    # Assuming you have a function to load documents (e.g., PyPDFLoader)
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    return docs

def summary_generation(file_path):
    # Load documents
    docs = load_documents(file_path)

    # Map
    map_template = """The following is a set of documents
    {docs}
    Based on this list of docs, please identify the main themes and concepts
    Expand the description of each topic and concept for 2-3 lines that should include its basic descriptions, key points, and formulas if any.
    Helpful Answer:"""
    map_prompt = PromptTemplate.from_template(map_template)
    map_chain = LLMChain(llm=llm, prompt=map_prompt)

    # Reduce
    reduce_template = """The following is a set of summaries:
    {docs}
    Take these and distill it into a final, consolidated summary of the main topics and concepts that should include definitions and formulas of the concepts. Mention all the key points and formulas related to a concept.
    Expand the description of each topic and concept for 2-3 lines.
    Helpful Answer:"""
    reduce_prompt = PromptTemplate.from_template(reduce_template)

    # Run chain
    reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

    # Takes a list of documents, combines them into a single string, and passes this to an LLMChain
    combine_documents_chain = StuffDocumentsChain(
        llm_chain=reduce_chain, document_variable_name="docs"
    )

    # Combines and iteratively reduces the mapped documents
    reduce_documents_chain = ReduceDocumentsChain(
        # This is the final chain that is called.
        combine_documents_chain=combine_documents_chain,
        # If documents exceed context for `StuffDocumentsChain`
        collapse_documents_chain=combine_documents_chain,
        # The maximum number of tokens to group documents into.
        token_max=4000,
    )

    # Combining documents by mapping a chain over them, then combining results
    map_reduce_chain = MapReduceDocumentsChain(
        # Map chain
        llm_chain=map_chain,
        # Reduce chain
        reduce_documents_chain=reduce_documents_chain,
        # The variable name in the llm_chain to put the documents in
        document_variable_name="docs",
        # Return the results of the map steps in the output
        return_intermediate_steps=False,
    )

    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=50
    )
    split_docs = text_splitter.split_documents(docs)
    all_summaries = map_reduce_chain.run(split_docs)
    print(all_summaries)

    return all_summaries



In [ ]:
all_summaries

In [ ]:
summary = summary_generation(r"C:\Users\MSHOME\Desktop\Newfolder\temp\inheritance-project\transcript_output.pdf")

-------------------------------------------------------------------------

In [ ]:
!pip install fpdf
!pip install pytube
!pip install youtube-transcript-api
!pip install fpdf
!pip install youtube_transcript_api

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
from fpdf import *

def get_transcript(youtube_url, output_pdf_path):
    video_id = youtube_url.split("v=")[-1]
    transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)

    # Try fetching the manual transcript
    try:
        transcript = transcript_list.find_manually_created_transcript()
        language_code = transcript.language_code  # Save the detected language
    except:
        # If no manual transcript is found, try fetching an auto-generated transcript in a supported language
        try:
            generated_transcripts = [trans for trans in transcript_list if trans.is_generated]
            transcript = generated_transcripts[0]
            language_code = transcript.language_code  # Save the detected language
        except:
            # If no auto-generated transcript is found, raise an exception
            raise Exception("No suitable transcript found.")

    full_transcript = " ".join([part['text'] for part in transcript.fetch()])

    # Save the transcript to a PDF file
    save_to_pdf(full_transcript, language_code, output_pdf_path)

    return full_transcript, language_code  # Return both the transcript and detected language

def save_to_pdf(transcript, language_code, output_pdf_path):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    pdf.cell(200, 10, txt=f"Language Code: {language_code}", ln=True, align='C')
    pdf.ln(10)

    pdf.multi_cell(0, 10, txt=transcript)

    pdf.output(output_pdf_path)

# Example usage
output_pdf_path = "transcript_output.pdf"
get_transcript("https://www.youtube.com/watch?v=LXkO4HdQXdo", output_pdf_path)

In [ ]:
!pip install nltk gensim spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import PyPDF2
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

In [ ]:
!pip install nltk

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    preprocessed_text = [lemmatizer.lemmatize(word.lower()) for word in word_tokens if word.isalnum() and word.lower() not in stop_words]
    return preprocessed_text


In [ ]:
from gensim import corpora, models

def perform_lda(preprocessed_text, num_topics=5):
    dictionary = corpora.Dictionary([preprocessed_text])
    corpus = [dictionary.doc2bow(preprocessed_text)]
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary)
    topics = lda_model.print_topics()
    return topics

In [ ]:
pdf_path = r"/content/transcript_output.pdf"
text = extract_text_from_pdf(pdf_path)
preprocessed_text = preprocess_text(text)
topics = perform_lda(preprocessed_text)
print(topics)


# BERT FINE-TUNED

In [ ]:
from transformers import TFAutoModel, AutoTokenizer

In [ ]:
model = TFAutoModel.from_pretrained('bert-base-uncased')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
input = tokenizer(text)

In [ ]:
%pip install rouge

In [ ]:
from rouge import Rouge

In [ ]:
# Your summaries and text
# Create Rouge object
rouge = Rouge()

# Calculate ROUGE scores
scores = rouge.get_scores(summary, text)

# Print ROUGE scores
print(scores)

In [ ]:
def calculate_rouge_score(reference, hypothesis):
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference)
    rouge_f1 = scores[0]['rouge-1']['p']
    return rouge_f1

In [ ]:
rouge_score = calculate_rouge_score(text, summary)
print(f"ROUGE-N Precision Score: {rouge_score}")